## Transformer и multi-head attention

План:
- посмотреть, как работает трансформер T5 на задаче первода с немецкого на английский с прошлой недели
- рассмотреть внутреннее устройство трансформера:
  - LayerNorm
  - Multi-head attention
  - Блоки энкодера и декодера
- попробовать обучить не хуже библиотечного

### 0. Забегая вперёд: T5

paper: https://arxiv.org/abs/1910.10683

In [1]:
import torch
from torch import Tensor, nn
from datasets import load_dataset, load_from_disk
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import lightning as L

from transformers import T5Tokenizer
from transformers import T5ForConditionalGeneration
from torch import nn, Tensor
import math
from typing import cast
from pathlib import Path

In [2]:
torch.manual_seed(42)

#### Датасет с прошлой практики

In [3]:
from lightning.pytorch.utilities.types import EVAL_DATALOADERS, TRAIN_DATALOADERS


class Multi30kDataset(L.LightningDataModule):
    train_dataset: Dataset
    test_dataset: Dataset
    tokenizer: T5Tokenizer

    def __init__(self, maxlen: int = 0, batch_size: int = 32) -> None:
        super().__init__()
        self.tokenizer = T5Tokenizer.from_pretrained(
            "t5-small", padding_size="right", bos_token="</b>", legacy=False
        )
        if not Path("bentrevett/multi30k-train").is_dir():
            self.train_dataset = load_dataset("bentrevett/multi30k", split="train")
            self.test_dataset = load_dataset("bentrevett/multi30k", split="test")
            self.train_dataset.save_to_disk("bentrevett/multi30k-train")
            self.test_dataset.save_to_disk("bentrevett/multi30k-test")
        else:
            self.train_dataset = load_from_disk("bentrevett/multi30k-train")
            self.test_dataset = load_from_disk("bentrevett/multi30k-test")

        self.batch_size = batch_size
        if maxlen > 0:
            self.train_dataset = self.filter_dataset(self.train_dataset, maxlen)
            self.test_dataset = self.filter_dataset(self.test_dataset, maxlen)

    def train_dataloader(self) -> TRAIN_DATALOADERS:
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            collate_fn=self.collate_fn,
        )

    def test_dataloader(self) -> EVAL_DATALOADERS:
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            collate_fn=self.collate_fn,
        )

    def collate_fn(self, batch: list[tuple[str, str]]) -> tuple[Tensor, Tensor]:
        prompt = self.tokenizer.bos_token
        inputs, targets = zip(*[(pair["de"], prompt + pair["en"]) for pair in batch])
        encoded_batch = self.tokenizer(
            inputs, text_target=targets, padding="longest", return_tensors="pt"
        )
        return encoded_batch

    @staticmethod
    def filter_dataset(dataset, maxlen: int) -> list[dict[str, str]]:
        return [
            dataset[i]
            for i in range(len(dataset))
            if len(dataset[i]["en"].split(" ")) <= maxlen
        ]

In [4]:
multi30k = Multi30kDataset(maxlen=7, batch_size=12)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Посмотрим, как быстро обучится T5 - одна из версий трансформера, без каких-либо рекуррентных блоков

In [5]:
t5 = T5ForConditionalGeneration.from_pretrained("t5-small")
# t5.init_weights()

Обернём в `LightningModule`

In [6]:
import lightning as L
from lightning.pytorch.utilities.types import STEP_OUTPUT, OptimizerLRScheduler


class Seq2Seq(L.LightningModule):
    def __init__(
        self,
        model: nn.Module,
        tokenizer: T5Tokenizer,
        lr: float = 0.01,
    ) -> None:
        super().__init__()
        self.model = model
        self.tokenizer = tokenizer
        self.lr = lr

    def training_step(self, batch: dict[str, Tensor], batch_idx: int) -> STEP_OUTPUT:
        if isinstance(self.model, T5ForConditionalGeneration):
            model_outputs = self.model.forward(input_ids=batch["input_ids"], labels=batch["labels"])
            loss = model_outputs.loss
        else:
            logits = self.model.forward(
                batch["input_ids"], batch["labels"]
            )
            loss = F.cross_entropy(
                logits[:, :-1].reshape(-1, len(self.tokenizer)),
                batch["labels"][:, 1:].flatten(),
            )
        self.log("loss", loss, prog_bar=True)
        return loss

    def configure_optimizers(self) -> OptimizerLRScheduler:
        return torch.optim.Adam(self.parameters(), lr=self.lr)

In [7]:
trainer = L.Trainer(accelerator="cpu", max_epochs=1, limit_train_batches=200, logger=False)
seq2seq = Seq2Seq(t5, multi30k.tokenizer, lr=0.001)
trainer.fit(model=seq2seq, train_dataloaders=multi30k.train_dataloader())

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/sarapulov/miniconda3/envs/dl-course/lib/python3.10/site-packages/lightning/pytorch/trainer/setup.py:201: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(
/Users/sarapulov/miniconda3/envs/dl-course/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:617: UserWarning: Checkpoint directory /Users/sarapulov/projects/dl-mkn/workshops/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 60.5 M
-----------------------------------------------------
60.5 M    Trainable params
0         Non-trainable params
60.5 M    Total par

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


Посмотрим на пример перевода:

In [8]:
batch = next(iter(multi30k.test_dataloader()))
preds = t5.generate(batch["input_ids"])

source, target, preds = map(
    lambda x: multi30k.tokenizer.batch_decode(x, skip_special_tokens=True),
    (batch["input_ids"], batch["labels"], preds),
)

for src, tgt, pred in zip(source, target, preds):
    print(f"Deutsch: {src}")
    print(f"English: {tgt}")
    print(f"Translation: {pred}\n")

/Users/sarapulov/miniconda3/envs/dl-course/lib/python3.10/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Deutsch: Ein Typ arbeitet an einem Gebäude.
English: A guy works on a building.
Translation: A man working on a building.

Deutsch: Drei Leute sitzen in einer Höhle.
English: Three people sit in a cave.
Translation: Three people sitting in a cave.

Deutsch: Leute, die vor einem Gebäude stehen.
English: People standing outside of a building.
Translation: People are standing in front of a building.

Deutsch: Ein Mann schneidet ste von Bäumen.
English: A man cutting branches of trees.
Translation: Man schneidet trees.

Deutsch: Ein Kind planscht im Wasser.
English: A child is splashing in the water
Translation: A child plans out of water.

Deutsch: Eine schöne Frau spielt auf einer Harfe.
English: A pretty woman plays a harpsichord.
Translation: A nice woman playing on a wall.

Deutsch: Leute sitzen in einem Zug.
English: People sit inside a train.
Translation: People sitting in a train.

Deutsch: Ein kleines Kind kocht mit einer anderen Person.
English: A toddler is cooking with another 

### 1. А теперь по порядку

Энкодер:
- начинаем с начальных эмбеддингов слов и их позиций (embedding / positional encoding)
- внутри блока энкодера каждый токен обменивается информацией с каждым, вес определяется механизмом внимания (self-attention)
- все токены обрабатываются параллельно

Декодер:
- начинаем с эмбеддинга текущего слова и его позиции (embedding / positional encoding)
- обновляем его механизмом внимания, но смотрим только на предыдущие слова (*masked self-attention*)
- обновляем эмбеддинг текущего токена вниманием на выход энкодера (*cross-attention*)
- движемся от начала предложения к концу последовательно

<img src="https://lena-voita.github.io/resources/lectures/seq2seq/transformer/transformer_original.gif" style="background:white" width="500"/>

Посмотрим на архитектуру в деталях

<img src="https://lilianweng.github.io/posts/2020-04-07-the-transformer-family/transformer.png" style="background:white" width="800"/>

- Input / Output Embedding - уже знакомый `nn.Embedding` слой
- Positional Encoding - было в практике VAE & DDPM (sinusoidal encoding), сегодня тоже возьмём `nn.Embedding`
- **Multi-Head [Masked] Attention** - сердце трансформера, будем детально разбирать
- Add & Norm - skip connection (знакомое) и **layer normalization** (посмотрим на отличие от batchnorm)
- Feed Forward - обычный перцептрон

#### 1.1. LayerNorm

$\begin{aligned}
y = \frac{x - \mathrm{E}[x]}{ \sqrt{\mathrm{Var}[x] + \epsilon}} * \gamma + \beta
\end{aligned}$

Но среднее и дисперсию считаем по-разному:
- BatchNorm: вдоль батча и пространственных (или временных) размерностей независимо для каждого канала
- LayerNorm: вдоль всех каналов отдельного примера
<!-- - BatchNorm: across batch size and spatial dimensions independently for each feature channel
- LayerNorm: across all channels for each individual sample -->

<!-- <img src="https://i.stack.imgur.com/fAowJ.png" style="background:white" width="800"/> -->

<img src="https://i.stack.imgur.com/7ZO1R.png" style="background:white" width="800"/>

In [9]:
batch, sentence_length, embedding_dim = 3, 3, 5
embedding = torch.randn(batch, sentence_length, embedding_dim)
var, mean = torch.var_mean(embedding, dim=[2], unbiased=False, keepdim=True)

layer_norm = nn.LayerNorm(embedding_dim, elementwise_affine=False)
assert torch.allclose(
    layer_norm(embedding), (embedding - mean) / torch.sqrt(var), rtol=1e-3
)

#### 1.2. Multi-head attention - главный компонент self-attention и cross-attention

Основная идея:
- На основе входных эмбеддингов рассчитываем векторы запросов $Q$, ключей $K$ и значений $V$
- Дополнительно предоставляется **маска внимания**: каким парам токенов разрешено смотреть друг на друга
- Мера релевантности каждого ключа нашему запросу - их **скалярное произведение**
- Новое [промежуточное] значение эмбеддинга - взвешенная по релевантности сумма значений
- Итоговый эмбеддинг - линейная проекция нескольких конкатенированных голов

Примечание: источником ключей и значений могут быть как общие данные (*self-attention*), так и различные (*cross-attention*)

<img src="https://www.researchgate.net/profile/A-X/publication/334427742/figure/fig3/AS:987861692211201@1612535989364/Structure-of-multi-head-attention-layer.ppm" style="background:white" height="350"/>

<img src="https://blog.sailor.plus/deep-learning/images/1613723693323.png" style="background:white" height="350"/>



In [10]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim: int, n_heads: int):
        super().__init__()

        assert hid_dim % n_heads == 0
        self.n_heads = n_heads

        # query, key and value
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)
        # out projection
        self.fc_o = nn.Linear(hid_dim, hid_dim)

    def forward(
        self, query: Tensor, key: Tensor, value: Tensor, mask: Tensor | None = None
    ) -> tuple[Tensor, Tensor]:
        B, T, C = query.shape
        H = self.n_heads

        # 1. получаем значения Q, K, V из линейных слоёв
        Q = self.fc_q.forward(query)  # B x T x C
        K = self.fc_k.forward(key)  # B x T x C
        V = self.fc_v.forward(value)  # B x T x C

        # 2. разбиваем результат на "головы" и выносим их размерность вперёд
        Q = Q.view(B, -1, H, C // H).permute(
            0, 2, 1, 3
        )  # B x T x C -> B x H x Lq x (C / H)
        K = K.view(B, -1, H, C // H).permute(
            0, 2, 1, 3
        )  # B x T x C -> B x H x Lk x (C / H)
        V = V.view(B, -1, H, C // H).permute(
            0, 2, 1, 3
        )  # B x T x C -> B x H x Lk x (C / H)

        # 3. рассчитываем матрицу внимания как скалярное произведение
        # между всеми парами токенов из Q и K для всех голов внимания
        scale = math.sqrt(C // H)
        attention = torch.matmul(Q, K.permute(0, 1, 3, 2)) / scale  # B x H x Lq x Lk

        # 4. получаем веса внимания для разрешённых токенов
        if mask is not None:
            attention = attention.masked_fill(mask == 0, float("-inf"))

        attention = torch.softmax(attention, dim=-1)  # B x H x Lq x Lk

        # 5. Получаем значения для всех голов внимания
        h = torch.matmul(attention, V)  # B x H x Lq x C / H

        # 6. конкатенируем головы и делаем финальную проекцию
        h = h.permute(0, 2, 1, 3).contiguous()  # B x Lq x H x (C / H)
        h = self.fc_o(h.flatten(2))  # B x Lq x H

        return h, attention

In [11]:
mha = MultiHeadAttentionLayer(128, 4)
print(sum([p.numel() for p in mha.parameters()]))
q = torch.randn(3, 5, 128)
k = torch.randn(3, 7, 128)
v = torch.randn(3, 7, 128)

h, attn = mha.forward(q, k, v)
print(h.shape)

66048
torch.Size([3, 5, 128])


#### 1.3. Блок энкодера

<img src="https://lilianweng.github.io/posts/2018-06-24-attention/transformer-encoder.png" width="400"/>
<!-- 
- LayerNorm
- SelfAttention
- LayerNorn
- FeedForward -->

In [12]:
class EncoderBlock(nn.Module):
    def __init__(self, hidden_dim: int, num_heads: int, fc_expand: int = 2) -> None:
        super().__init__()
        self.layer_norm_1 = nn.LayerNorm(hidden_dim)
        self.layer_norm_2 = nn.LayerNorm(hidden_dim)
        self.mha = MultiHeadAttentionLayer(hidden_dim, num_heads)
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim * fc_expand),
            nn.ReLU(),
            nn.Linear(hidden_dim * fc_expand, hidden_dim),
        )

    def forward(self, h: Tensor, mask: Tensor | None = None) -> Tensor:
        # self-attention, skip connection, layernorm
        ...
        # feed-forward, skip connection, layernorm
        ...
        return h

In [13]:
encoder = EncoderBlock(128, 4)
print(sum([p.numel() for p in encoder.parameters()]))

h = torch.randn(3, 7, 128)

h = encoder.forward(h)
print(h.shape)

132480
torch.Size([3, 7, 128])


Созданим модуль для энкодера с произвольным количеством блоков.

Внутри него будем:
- Получать эмбеддинг из токенов и позиций
- Обновлять эмбеддинги серией блоков энкодера

Пример модели, состоящей только из энкодер-блоков: BERT

In [14]:
class Encoder(nn.Module):
    def __init__(
        self,
        vocab_size: int,
        hidden_dim: int,
        n_layers: int,
        n_heads: int,
        max_length: int = 100,
    ) -> None:
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, hidden_dim)
        self.position_embedding = nn.Embedding(max_length, hidden_dim)
        self.layers = nn.ModuleList(
            [EncoderBlock(hidden_dim, n_heads) for _ in range(n_layers)]
        )

    def forward(self, src: Tensor, src_mask: Tensor) -> Tensor:
        B, T = src.shape
        # тензор с номерами позиций
        pos = torch.arange(0, T).unsqueeze(0).repeat(B, 1).to(src.device)
        
        # стартовый эмбеддинг: сумма эмбеддинга токена и эмбеддинга позиции
        h = self.token_embedding(src) + self.position_embedding(pos)

        for layer in self.layers:
            h = layer(h, src_mask)
        return h

In [15]:
src = torch.randint(0, 20, size=(3, 5))
encoder = Encoder(vocab_size=20, hidden_dim=8, n_heads=2, n_layers=2)
encoded = encoder.forward(src, None)
print(encoded.shape)

torch.Size([3, 5, 8])


#### 1.4. Блок декодера

То же самое, два отличия:
- в блоке self-attention можно смотреть только на предыдущие токены
- добавлен блок cross-attention для получения информации из декодера

<img src="https://lilianweng.github.io/posts/2018-06-24-attention/transformer-decoder.png" width="400"/>

<!-- |<img src="https://lilianweng.github.io/posts/2018-06-24-attention/transformer-decoder.png" width="300"/> |$E = mc^2$|
|-|-| -->


In [16]:
class DecoderBlock(nn.Module):
    def __init__(self, hidden_dim: int, num_heads: int, fc_expand: int = 2) -> None:
        super().__init__()
        self.layer_norm_1 = nn.LayerNorm(hidden_dim)
        self.layer_norm_2 = nn.LayerNorm(hidden_dim)
        self.layer_norm_3 = nn.LayerNorm(hidden_dim)
        self.self_attention = MultiHeadAttentionLayer(hidden_dim, num_heads)
        self.cross_attention = MultiHeadAttentionLayer(hidden_dim, num_heads)
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim * fc_expand),
            nn.ReLU(),
            nn.Linear(hidden_dim * fc_expand, hidden_dim),
        )

    def forward(
        self,
        h: Tensor,
        tgt_mask: Tensor | None,
        input_embeds: Tensor,
        src_mask: Tensor | None,
    ) -> Tensor:

        # masked self-attention, skip connection, layernorm
        ...

        # cross-attention, skip connection, layernorm
        ...

        # feed-forward, skip connection, layernorm
        ...
        
        return h

In [17]:
decoder = DecoderBlock(128, 4)
print(sum([p.numel() for p in encoder.parameters()]))

tgt = torch.randn(3, 9, 128)

tgt = decoder.forward(tgt, None, h, None)
print(tgt.shape)

2160
torch.Size([3, 9, 128])


Созданим модуль для декодера с произвольным количеством блоков.

Внутри него будем:
- Получать эмбеддинг из токенов и позиций (как в энкодере)
- Обновлять эмбеддинги серией блоков декодера
- Получать логиты над словарём из эмбеддингов

Пример модели, состоящей только из декодер-блоков: GPT

In [18]:
class Decoder(nn.Module):
    def __init__(
        self,
        vocab_size: int,
        hidden_dim: int,
        n_layers: int,
        n_heads: int,
        max_length: int = 100,
    ) -> None:
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, hidden_dim)
        self.position_embedding = nn.Embedding(max_length, hidden_dim)
        self.layers = nn.ModuleList(
            [
                DecoderBlock(
                    hidden_dim,
                    n_heads,
                )
                for _ in range(n_layers)
            ]
        )
        self.fc_out = nn.Linear(hidden_dim, vocab_size)

    def forward(
        self, tgt: Tensor, encoded_src: Tensor, tgt_mask: Tensor, src_mask: Tensor
    ) -> Tensor:
        B, T = tgt.shape
        # тензор с номерами позиций
        pos = torch.arange(0, T).unsqueeze(0).repeat(B, 1).to(tgt.device)

        # стартовый эмбеддинг: сумма эмбеддинга токена и эмбеддинга позиции
        tgt = self.token_embedding(tgt) + self.position_embedding(pos)

        for layer in self.layers:
            layer = cast(DecoderBlock, layer)
            tgt = layer.forward(tgt, tgt_mask, encoded_src, src_mask)

        # логиты
        output = self.fc_out(tgt)

        return output

Проверяем:

In [19]:
decoder = Decoder(vocab_size=20, hidden_dim=8, n_layers=2, n_heads=2)
tgt = torch.randint(0, 20, size=(3, 7))
logits = decoder.forward(tgt, encoded, None, None)
logits.shape

torch.Size([3, 7, 20])

### 2. Собираем трансформер

Примечание: трансформером теперь называют не только encoder-decoder модели (vanilla transformer, T5) но и encoder-only модели (BERT), и decoder-only модели (GPT)

<img src="https://user-images.githubusercontent.com/49787234/103397155-ff354180-4b71-11eb-8283-1c0f50f5b462.jpg" width="800"/>

In [20]:
class Transformer(nn.Module):
    def __init__(
        self,
        encoder,
        decoder,
        pad_token_id,
    ):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.pad_token_id = pad_token_id

    def make_src_mask(self, src: Tensor):
        src_mask = (src != self.pad_token_id).unsqueeze(1).unsqueeze(2)  # B x 1 x 1 x T
        return src_mask

    def make_tgt_mask(self, tgt: Tensor):
        B, T = tgt.shape
        tgt_pad_mask = (
            (tgt != self.pad_token_id).unsqueeze(1).unsqueeze(2)
        )  # B x 1 x 1 x T
        tgt_sub_mask = torch.tril(torch.ones((T, T), device=tgt.device)).bool()  # T x T
        tgt_mask = tgt_pad_mask & tgt_sub_mask  # B x 1 x T x T
        return tgt_mask

    def forward(self, src: Tensor, tgt: Tensor) -> Tensor:
        # строим маски для энкодера и декодера
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_tgt_mask(tgt)

        encoded_src = self.encoder(src, src_mask)
        logits = self.decoder(tgt, encoded_src, trg_mask, src_mask)

        return logits

    def generate(
        self, src: Tensor, bos_token_id: int, max_new_tokens: int = 20
    ) -> Tensor:
        idx = torch.full((src.shape[0], 1), fill_value=bos_token_id)
        for t in range(max_new_tokens):
            logits = self.forward(src, idx)[:, -1]
            new_token = logits.argmax(dim=-1, keepdim=True)
            idx = torch.cat([idx, new_token], dim=1)

        return idx

In [21]:
src = torch.randint(0, 20, size=(3, 5))
tgt = torch.randint(0, 20, size=(3, 9))

encoder = Encoder(
    vocab_size=len(multi30k.tokenizer), hidden_dim=256, n_layers=1, n_heads=4
)
decoder = Decoder(
    vocab_size=len(multi30k.tokenizer), hidden_dim=256, n_layers=1, n_heads=4
)
transformer = Transformer(encoder, decoder, pad_token_id=0)

print(sum([p.numel() for p in transformer.parameters()]))
print(transformer.forward(src, tgt).shape)

26054757
torch.Size([3, 9, 32101])


### 3. Пробуем обучить

In [22]:
torch.manual_seed(42)
encoder = Encoder(
    vocab_size=len(multi30k.tokenizer), hidden_dim=256, n_layers=1, n_heads=4
)
decoder = Decoder(
    vocab_size=len(multi30k.tokenizer), hidden_dim=256, n_layers=1, n_heads=4
)
transformer = Transformer(encoder, decoder, pad_token_id=0)
seq2seq = Seq2Seq(transformer, multi30k.tokenizer, lr=0.0005)

In [23]:
trainer = L.Trainer(accelerator="cpu", max_epochs=6, limit_train_batches=200, logger=False)
trainer.fit(model=seq2seq, train_dataloaders=multi30k.train_dataloader())

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/sarapulov/miniconda3/envs/dl-course/lib/python3.10/site-packages/lightning/pytorch/trainer/setup.py:201: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(

  | Name  | Type        | Params
--------------------------------------
0 | model | Transformer | 26.1 M
--------------------------------------
26.1 M    Trainable params
0         Non-trainable params
26.1 M    Total params
104.219   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=6` reached.


In [24]:
def translate_batch(
    batch: dict[str, Tensor], model: Transformer, tokenizer: T5Tokenizer
):
    source = batch["input_ids"]
    target = batch["labels"]

    preds = model.generate(source, tokenizer.bos_token_id, max_new_tokens=20)

    # decode

    source, target, preds = map(
        lambda x: tokenizer.batch_decode(x, skip_special_tokens=True),
        (source, target, preds),
    )

    for src, tgt, pred in zip(source, target, preds):
        print(f"Deutsch: {src}")
        print(f"English: {tgt}")
        print(f"Translation: {pred}\n")

In [25]:
translate_batch(next(iter(multi30k.test_dataloader())), transformer, multi30k.tokenizer)

Deutsch: Ein Typ arbeitet an einem Gebäude.
English: A guy works on a building.
Translation: A guy working on a building.

Deutsch: Drei Leute sitzen in einer Höhle.
English: Three people sit in a cave.
Translation: Three people are sitting in a field.

Deutsch: Leute, die vor einem Gebäude stehen.
English: People standing outside of a building.
Translation: People are standing outside in a building.

Deutsch: Ein Mann schneidet ste von Bäumen.
English: A man cutting branches of trees.
Translation: A man is cutting a green slide.

Deutsch: Ein Kind planscht im Wasser.
English: A child is splashing in the water
Translation: A child splashes in the water.

Deutsch: Eine schöne Frau spielt auf einer Harfe.
English: A pretty woman plays a harpsichord.
Translation: A lovely woman playing on a wave.

Deutsch: Leute sitzen in einem Zug.
English: People sit inside a train.
Translation: People sitting in a train.

Deutsch: Ein kleines Kind kocht mit einer anderen Person.
English: A toddler is c